In [3]:
import itertools

from numpy import loadtxt
from tensorflow.keras.models import load_model
import pickle
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
# model_name="resnet34-1632051334.h5"
model = load_model('C:/Users/moham/Desktop/MyDeepLearningProject/resnet34-1632051334.h5', compile=False)
# model =load_model(model_name)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, None, None,  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d[0][0]             
____________________________________________________________________________________________

In [2]:
def plotImages(images_arr):
    fig,axes = plt.subplots(1,10, figsize=(20,20))
    axes = axes.flatten()
    for img,ax in zip(images_arr,axes):
        ax.imshow(img,'gray')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
dirpath='C:/Users/moham/Desktop/Data/breastImagesJPEG'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    dirpath+'/test',
    target_size=(250, 250),
    batch_size=10,
    shuffle=False
)
test_images, test_labels = next(test_generator)
plotImages(test_images)
print (test_labels)

In [ ]:
test_generator.classes

In [ ]:
predictions = model.predict(x=test_generator,verbose=0)

In [ ]:
np.round(predictions)

In [ ]:
cm=confusion_matrix(y_true=test_generator.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
def plot_confusion_matrix(cm,classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
	plt.imshow(cm, interpolation='nearest', cmap=cmap)
	plt.title(title)
	plt.colorbar()
	tick_marks = np.arange(len(classes))
	plt.xticks(tick_marks,classes, rotation=45)
	plt.yticks(tick_marks,classes)

	if normalize:
		cm = cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
		print("Normalized Confusion matrix")
	else:
		print("Confusion matrix, without normalization")

	print(cm)

	thresh = cm.max()/ 2.
	for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
		plt.text(j,i,cm[i,j],
				 horizontalalignment="center",
				 color="white" if cm[i,j] > thresh else "black")

	plt.tight_layout()
	plt.ylabel('true label')
	plt.xlabel('predicted label')

In [ ]:
CAT=['Benign','Cancer','Normal']
cm_plot_labels = CAT
plot_confusion_matrix(cm=cm,classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
score = model.evaluate(test_generator,verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_true=test_generator.classes,y_pred=np.argmax(predictions, axis=-1),digits=3))